<h1 style="text-align:center">Modern Gaussian Processes: Scalable Inference and Novel Applications (IJCNN '19)</h1>
<h2 style="text-align:center">Tutorial: Inference of Gaussian Process Regression models</h2>

- **Edwin V. Bonilla**, Data61, Australia 
- **Maurizio Filippone**, EURECOM, France

## 1. Aims
<div class="alert alert-info">
<ul> 
<li> To implement Gaussian process inference for regression.
<li> To use the above to observe samples from a Gaussian process posterior distribution.
<li> To evaluate how different hyperparameter settings impact model quality.
<li> To investigate different kernel functions and parameter optimisation strategies.
</ul>
</div>

**Note**: we shall use PyTorch for reasons that will be clear later on in this notebook

In [ ]:
import torch
import re
if int(re.search(r'([\d.]+)', torch.__version__).group(1).replace('.', '')) < 100:
    raise ImportError('Your PyTorch version is not supported. Please download and install PyTorch 1.x')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import warnings
matplotlib.rc_file('matplotlibrc')
warnings.filterwarnings('ignore')

def set_seed(seed=0):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

## 2. Introduction
<div class="alert alert-info">
In this tutorial, we shall cover the basic concepts of <b>GP regression</b>. For the sake of clarity, we shall focus on univariate data, which allows for better visualisation of the GP model. Nonetheless, the code implemented within this lab can be very easily extended to handle
multi-dimensional inputs.
</div>

<div class="alert alert-info">
As in the previous notebook, we shall consider a one-dimensional regression problem, whereby the inputs x are transformed by
a function 
<br><br>
    $$ f(\mathbf{x}) = sin(exp(0.03 * \mathbf{x}))\,.$$
<br>
Let's generate 500 random points, $x$, in the range $[-20, 80]$, and compute their corresponding function
values, $y$ (assuming noisyless observations for the moment). The target function can then be plotted accordingly.
</div>

In [ ]:
set_seed()
matplotlib.rc_file('matplotlibrc')
n = 100

# Define our function
f = lambda x: np.sin(np.exp(.03 * x))
# Define our observation points
x = np.sort(np.random.uniform(-20,80,n))
# Define our target points
y = f(x)

# Plot 
fig, ax = plt.subplots()
ax.plot(x, y, '.', c='black', label='Observations')
ax.plot(np.linspace(-30, 85, 1000), f(np.linspace(-30, 85, 1000)), alpha=0.6, label=r"$f({x}) = sin(e^{0.03 {x}})$" )
ax.set_title("Regression problem")
ax.set_ylim(-1.5, 1.5)
ax.legend()
plt.show()

Recall that GPs are non-parametric model. We define a prior distribution over functions (models),
specified as a multivariate Gaussian distribution $p(f) = N (\mu, \Sigma)$.

Without loss of generality, we shall assume a zero-mean GP prior, i.e. $\mu = 0$. The covariance
matrix of the distribution, $\Sigma$, may then be computed by evaluating the covariance between the
input points. For this tutorial, we consider the widely used squared-exponential (RBF)
covariance.

As a reminder, the RBF kernel is defined between two points as: 

$$k(x, x') = \sigma_f^2 \exp \Big( -\dfrac {(x-x')^2}{2l^2} \Big). $$

This kernel is parameterised by a lengthscale parameter $l$, and variance $\sigma_f^2$ . Given that the true
function may be assumed to be corrupted with noise, we can also add a noise parameter, $\sigma_n^2$ , to
the diagonal entries of the resulting kernel matrix, $K$, such that
$$K_y = K + \sigma_n^2I.$$
<br>

In [ ]:
@torch.jit.script
def cdist(x1, x2):
    """
    Compute distance between each pair of the two collections of input tensors.
    see scipy.spatial.distance.cdist
    """
    x1_norm = x1.pow(2).sum(dim=-1, keepdim=True)
    x2_norm = x2.pow(2).sum(dim=-1, keepdim=True)
    res = torch.addmm(x2_norm.transpose(-2, -1), x1, x2.transpose(-2, -1), alpha=-2).add_(x1_norm)
    res = res.clamp_min_(1e-30).sqrt_()
    return res

def rbf_kernel(x1, x2, lengthscale, variance):
    """
    Compute the RBF covariance matrix 
    """
    K = variance * torch.exp(-cdist(x1[...,None], x2[...,None])**2 / (2 * lengthscale**2))
    return K

Assuming a zero-mean prior, and using the kernel matrix constructed with `rbf_kernel()` for input points x, we can sample from the prior distribution using the numpy `multivariate_normal()` function.
<br><br>
For the time being, you can initialise the kernel parameters as follows:
<br>
- lengthscale = 10<br>
- variance = 0.1

In [ ]:
set_seed()
# Define points 
test_x = torch.from_numpy(np.linspace(-30, 85, 200)).float()

# Define kernel parameters
lengthscale = 10
variance = .1

# Sample from GP prior
mu = np.zeros(len(test_x))
K = rbf_kernel(test_x, test_x, lengthscale, variance)
samples = 30
f_i = np.random.multivariate_normal(mu, K, samples)

# Plot
fig, ax = plt.subplots()
for i in range(samples):
    ax.plot(test_x.numpy(), f_i[i,:], c='#ff7f00', alpha=0.5)

ax.plot(test_x.numpy(), mu, color="grey", label=r'prior $\mu$')   
ax.fill_between(test_x.numpy(),mu + np.sqrt(variance) * 2, mu - np.sqrt(variance) * 2,
                color="grey", alpha=0.2, label=r'prior $2\sigma\approx95\%\,CI$')
ax.set_title('Sampling from the GP prior')
ax.set_ylim(-1.5, 1.5)
ax.legend()
plt.show()

## 3. GP Inference
<div class="alert alert-info">
Suppose we can now observe 3 points at random from the input data; we would expect that with this additional knowledge, the functions drawn from the updated GP distribution would be constrained to pass through these points (or at least close if corrupted with noise). The combination of the prior and the likelihood of the observed data leads to the posterior distribution over functions.
<br><br>
Assign 3 points at random from $x$ (and their corresponding function values) to `obs_x` and `obs_t`
respectively. For now we shall assume that all other $x$ values are unobserved.<br><br>

You are encouraged to use the given initial configuration.
</div>

<div class="alert alert-info">

A complete implementation of `gp_inference` is provided for evaluating the posterior GP mean and variance using the equations given in the tutorial.
<br><br>
<b>Note</b>: Matrix inversions can be both numerically troublesome and slow to compute. In this notebook, we shall avoid computing matrix inversions directly by instead considering Cholesky decompositions for solving linear systems. You are encouraged to read more about Cholesky decompositions for GPs by consulting Appendix A.4 of <a target="_blank" href="http://www.gaussianprocess.org/gpml/">Gaussian Processes for Machine Learning (Rasmussen and Williams, 2005)</a> - available online!<br><br>
The complete pseudo-code for the following procedure is provided in Algorithm 2.1 from Chapter 2 of this same book.
</div>

In [ ]:
def gp_inference(obs_x, obs_t, x_new, params, prior_mean=0):
    
    # unpack params
    lengthscale = params[0]
    variance = params[1]
    noise = params[2]
    obs_t = obs_t - prior_mean
    
    N = obs_x.shape[0]
    
    # compute kernel
    K = rbf_kernel(obs_x, obs_x, lengthscale,variance)
    K_y = K + noise * torch.eye(obs_x.shape[0])
     
    '''
    When computing the posterior mean, we would like to avoid evaluating
        
                            alpha = (K_y)^-1 * obs_t
    
    directly. The Cholesky decomposition can be applied using the following procedure.
    
        -> Compute the lower triangular Cholesky decomposition of K_y (which we shall call K_chol)
        -> Compute 'alpha' as:
        
                            alpha = K_chol.T \ (K_chol \ obs_t)
                            
           where the back-substitution operator can be evaluated using the 'trtrs' (solve_triangular) 
           function in pytorch. Make sure to set the function's upper' flag as appropriate.
    '''  
    # compute the Cholesky decomposition of K_y
    K_chol = torch.cholesky(K_y)
    
    # compute alpha
    alpha = torch.trtrs(torch.trtrs(obs_t, K_chol, upper=False)[0], K_chol.t(), upper=True)[0]
    
    # compute the covariance between the training and test data
    K_obs_pred = rbf_kernel(obs_x, x_new, lengthscale, variance)
    
    # compute the covariance for the test data
    K_pred = rbf_kernel(x_new, x_new, lengthscale, variance)
    
    # compute the posterior mean
    posterior_m = torch.matmul(K_obs_pred.t(), alpha)
    
    '''
    Similarly, when computing
    
                        v = (kern_obs)^-1 * kern_obs_pred
                        
    employ the Cholesky decomposition as outlined above.                 
    '''
    # compute the posterior variance
    v = torch.trtrs(torch.trtrs(K_obs_pred, K_chol,upper=False)[0], K_chol.t(),upper=True)[0]
    posterior_v =  K_pred - torch.matmul(K_obs_pred.t(), v)
    
    # compute the marginal log-likelihood
    log_lik = -.5 * (torch.sum(torch.log(torch.abs(torch.diag(K_chol)))) +
                     torch.sum(torch.trtrs((obs_t), K_chol,upper=False)[0]**2)) - N/2. * np.log(2 * np.pi)
    
    return posterior_m[...,0] + prior_mean, posterior_v, log_lik

Now we can run the GP inference. For plot convenience, let's take 1000 points on the real axis as $x_\mathrm{new}$.

In [ ]:
set_seed()

# Define observed points
numObs = 3
obs_x = torch.from_numpy(np.sort((np.random.choice(x, numObs, replace=False)))).float()
obs_t = f(obs_x)

# Define kernel parameters
lengthscale = 10
variance = .1
noise = 1e-5
params = [lengthscale, variance, noise]

# Run inference and get posterior mean and variance
posterior_m, posterior_v, log_lik = gp_inference(obs_x, obs_t, test_x, params)
posterior_std = torch.sqrt(torch.diag(posterior_v))

## 4. Sampling from the  GP Posterior
<br>
<div class="alert alert-info">
Now that you have computed the posterior mean and variance, let's create a figure showing the true function. To this figure, we add the posterior mean and uncertainty (show two standard deviations) evaluated on the same $x$ values. Remember that the variance at each point is given by the diagonal of the covariance matrix.
    Let's also plot 10 samples from the posterior GP after inference.
<!-- Recall that the standard deviation is the square root of the variance. -->
</div>

In [ ]:
set_seed()

# Samples from the GP (posterior in this case)
samples = 10
f_i = np.random.multivariate_normal(posterior_m, posterior_v, samples) 

# Plot
fig, ax = plt.subplots()
ax.plot(test_x.numpy(), f(test_x).numpy(), alpha=0.75, label=r"$f({x}) = sin(e^{0.03 {x}})$" )

for i in range(samples):
    ax.plot(test_x.numpy(),f_i[i,:], c="#ff7f00", alpha=0.5)
ax.plot(obs_x.numpy(), obs_t.numpy(), '.', c='black', label='Observations')

ax.plot(test_x.numpy(), posterior_m.numpy(), color="grey", label=r'posterior $\mu$')  

ax.fill_between(test_x.numpy(), (posterior_m + posterior_std * 2).numpy(), (posterior_m - posterior_std * 2).numpy(),
                color="grey", alpha=0.2, label=r'posterior $2\sigma\approx95\%\,CI$')

ax.set_title('Sampling from the GP posterior')
ax.legend()
ax.set_ylim(-1.5, 1.5)
plt.show()

<div class="alert alert-info">
Try to increase the number of observations. What do you see?
</div>


In [ ]:
set_seed()
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2, sharex=True, sharey=True)

# Just a simple function to make it less verbose
def run_inference(numObs, ax):
    obs_x = torch.from_numpy(np.sort((np.random.choice(x, numObs, replace=False)))).float()
    obs_t = f(obs_x)

    posterior_m, posterior_v, log_lik = gp_inference(obs_x, obs_t, test_x, params) 
    posterior_std = torch.sqrt(torch.diag(posterior_v))
    
    ax.plot(obs_x.numpy(), obs_t.numpy(), '.', c='black', label='Observations')
    ax.plot(test_x.numpy(), posterior_m.numpy(), color="grey", label=r'posterior $\mu$')  
    ax.fill_between(test_x.numpy(), (posterior_m + posterior_std * 2).numpy(), (posterior_m - posterior_std * 2).numpy(),
                    color="grey", alpha=0.2, label=r'posterior $2\sigma\approx95\%\,CI$')
  
    ax.set_title('%d observations' % numObs)    
    ax.set_ylim(-1.5, 1.5)
    
    
run_inference(4, ax0)
run_inference(10, ax1)
run_inference(15, ax2)
run_inference(35, ax3)

ax3.legend(loc='lower left')
fig.suptitle('GP inference with increasing training points', y=1.02)
plt.show()

<h2> 4.1 Model evaluation</h2>
<div class="alert alert-info">
Try to change the GP prior (for instance the lenghtscale of the RBF kernel) and run again the GP inference. What do you see?
</div>

In [ ]:
set_seed()

# Define points
numObs =  15 
obs_x = torch.from_numpy(np.sort((np.random.choice(x, numObs, replace=False)))).float()
obs_t = f(obs_x)

# Just a simple function to make it less verbose
def run_inference(params, ax):
    posterior_m, posterior_v, log_lik = gp_inference(obs_x, obs_t, test_x, params) 
    posterior_std = torch.sqrt(torch.diag(posterior_v))
    
    ax.plot(obs_x.numpy(), obs_t.numpy(), '.', c='black', label='Observations')
    ax.plot(test_x.numpy(), posterior_m.numpy(), color="grey", label=r'posterior $\mu$')  
    ax.fill_between(test_x.numpy(), (posterior_m + posterior_std * 2).numpy(), (posterior_m - posterior_std * 2).numpy(),
                    color="grey", alpha=0.2, label=r'posterior $2\sigma\approx95\%\,CI$')
    ax.set_title(r'$l=$%.0f, $\sigma_f^2=$%.2f' % (params[0], params[1]))
    ax.text(80, 1.1, r'$p(Y|X, \theta)$ = %.2f' % log_lik, horizontalalignment='right')
    ax.set_ylim(-1.5, 1.5)

fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2, sharex=True, sharey=True)

# params = [lengthscale, variance, noise]
params = [3, 0.01, 1e-2]
run_inference(params, ax0)
params = [3, 0.1, 1e-2]
run_inference(params, ax1)
params = [10, 0.01, 1e-2]
run_inference(params, ax2)
params = [10, 0.1, 1e-2]
run_inference(params, ax3)

ax3.legend(loc='lower left')
fig.suptitle('Inference with different GP prior', y=1.02)
plt.show()

<div class="alert alert-info">
As a measure of model quality, you should check the log marginal likelihood of the model (the higher, the better).
<!-- To this end, complete the code provided in `gp inference()` to include the negative log likelihood term. -->
<br><br>
You could attempt a grid search over a range of parameter values in order to determine which configuration yields the best result<br><br>
</div>

In [ ]:
set_seed()

# Define points
numObs =  15 
obs_x = torch.from_numpy(np.sort((np.random.choice(x, numObs, replace=False)))).float()
obs_t = f(obs_x)

# Plot the likelihood surface
l_points = np.linspace(7, 10, 50)
logsigmaf_points = np.logspace(-2, 0, 50, base=10)
log_lik_points = np.zeros([len(l_points), len(logsigmaf_points)])

for i, logl in enumerate(l_points):
    for j, logsigmaf in enumerate(logsigmaf_points):
        posterior_m, posterior_v, log_lik = gp_inference(obs_x, obs_t, 
                                                         torch.from_numpy(np.linspace(-30, 85, 1)).float(),
                                                         [logl, 10. ** logsigmaf, 1e-5]) 
        log_lik_points[i, 49-j] = -log_lik
    
fig, ax = plt.subplots()   
cp = ax.contour(logsigmaf_points, l_points, log_lik_points, levels=np.logspace(np.log(6.5), np.log(9), 30, base=np.e))
cb = plt.colorbar(cp)
cb.ax.set_title(r"$p(Y|X, \theta)$",)   
best_coordinates = np.unravel_index(log_lik_points.argmin(), log_lik_points.shape)
ax.plot(logsigmaf_points[best_coordinates[1]], l_points[best_coordinates[0]], '*', markersize=16, label='Best hyperparameters')
ax.set_ylabel(r'lenghtscale l')
ax.set_xlabel(r'signal variance $\sigma_f^2$')
ax.legend(loc='bottom left')
ax.set_title('Optimization landscape\nof the negative marginal likelihood')
ax.semilogx()
plt.show()

But can we do better?

## 5. Parameter Optimisation using Gradient Descent
<br>
<div class="alert alert-info">
Optimise the hyperparameters of the model by minimising the negative log-likelihood of the model. For a complete solution, you should include the derivatives of the objective function with respect to the parameters being optimised.
<br><br>
The general formula for computing the derivative is given below:<br>
$$
\frac{\partial\;\text{NLL}}{\partial\;\theta_i} = - \frac{1}{2} \textbf{Tr} \left( K^{-1} \frac{\partial K}{\partial \theta_i} \right) + \frac{1}{2} \textbf{y}^{T} K^{-1} \frac{\partial K}{\partial \theta_i} K^{-1} \textbf{y}.
$$<br>
To give a more concrete example, the $\frac{\partial K}{\partial \theta_i}$ term for the lengthscale parameter in the RBF kernel is computed as follows:
$$
\frac{\partial K}{\partial l} = \sigma_f^2 \exp \left( -\dfrac {(x-x')^2}{2l^2} \right)\left( \dfrac {(x-x')^2}{l^3} \right)
$$
<br><br>
<b>Pro tip:</b> Note that the parameters $l$, $\sigma_f^2$ , and $\sigma_n^2$ are always expected to be positive. It is possible that the optimisation algorithm attempts to evaluate the log-likelihood in regions of the parameter space where one or more of these parameters are negative, leading to numerical issues. A commonly-used technique to enforce this condition is to work with a transformed version of covariance parameters using the logarithm transformation. In particular, define $\psi_l = log(l)$, $\psi_f = log(\sigma_f^2 )$, and $\psi_n = log(\sigma_n^2 )$, and optimise with respect to the $\psi$ parameters. The optimisation problem in the transformed space is now unbounded, and the gradient of the log-likelihood should be computed with respect to the $\psi$ parameters.
<br><br>
<b>Pro tip 2019:</b>
We don't really need to derive the gradients of the marginal likelihood w.r.t. parameters by hand. We can leverage the automatic differentiation engine in PyTorch! All the operations that we used (cdist and the RBF kernel function, the Cholesky decomposition and triangular linear system solver are all differentiable). Let's take advantage of that!
</div>

In [ ]:
set_seed()

# Transformed covariance parameters
logl = torch.nn.Parameter(torch.tensor(2).float())
logsigmaf = torch.nn.Parameter(torch.tensor(-2).float())
logsigman = torch.nn.Parameter(torch.tensor(-2).float())

# Just like vanilla PyTorch for DL
optimizer = torch.optim.SGD([logl, logsigmaf, logsigman], lr=0.01, momentum=0.95, nesterov=True)

# Define points
numObs =  15 
obs_x = torch.from_numpy(np.sort((np.random.choice(x, numObs, replace=False)))).float()
obs_t = f(obs_x)

# Run training loop
nlog_lik_steps = []
for i in range(200):
    optimizer.zero_grad()
    params = [logl.exp(), logsigmaf.exp(), logsigman.exp()]
    _, _, log_lik = gp_inference(obs_x, obs_t, torch.from_numpy(np.linspace(-30, 85, 1)).float(), params) 
    
    # Instead for maximize the log-likelihood, we minimize the negative log likelihood
    nlog_lik = -log_lik
    nlog_lik.backward()
    optimizer.step()
    nlog_lik_steps.append(nlog_lik)
    
print('NLOG_LIK = %.2f, LENGHTSCALE = %.2f, SIGMA_F = %.2f, SIGMA_N = %.2f' % (nlog_lik.item(), logl.exp(), logsigmaf.exp(), logsigman.exp()))

Did it converge?

In [ ]:
fig, ax = plt.subplots(figsize=[8, 3])
ax.plot(range(len(nlog_lik_steps)), nlog_lik_steps)
ax.set_ylabel(r'$p(Y|X, \theta^t)$')
ax.set_xlabel('Step')
ax.set_title('Optimization of the marginal likelihood w.r.t the kernel parameters')
plt.show()

Let's see where and how it converged!

In [ ]:
params = [logl.exp(), logsigmaf.exp(), logsigman.exp()]

# Run inference and ...
posterior_m, posterior_v, log_lik = gp_inference(obs_x, obs_t, test_x, params) 
posterior_m = posterior_m.detach()
posterior_v = posterior_v.detach()
posterior_std = torch.sqrt(torch.diag(posterior_v))

# ... and plot
fig, ax = plt.subplots()
ax.plot(obs_x.numpy(), obs_t.numpy(), '.', c='black', label='Observations')
ax.plot(test_x.numpy(), posterior_m.numpy(), color="grey", label=r'posterior $\mu$')  
ax.fill_between(test_x.numpy(), (posterior_m + posterior_std * 2).numpy(), (posterior_m - posterior_std * 2).numpy(),
                    color="grey", alpha=0.2, label=r'posterior $2\sigma\approx95\%\,CI$')
ax.set_title(r'$l=$%.2f, $\sigma_f^2=$%.2f, $\sigma_n^2=$%.2f' % (params[0], params[1], params[2]))
ax.text(80, 1.25, r'$p(Y|X, \theta)$ = %.2f' % log_lik, horizontalalignment='right')
ax.legend()
ax.set_ylim(-1.5, 1.5)
plt.show()

Interesting! During optimization of the marginal likelihood, the model recovered that observations were noisyless. 

That's it, folks!

# Bonus: Regression on Classification labels

Classification using non-Gaussian likelihood makes inference intractable. 
One could think to solve this issue by running GP regression directly on classification labels. 
In this case each point would be associated with a Gaussian likelihood, which is not the appropriate noise model for Bernoulli-distributed variables. 
Recently it has been shown how to transform the labels in a latent space where a Gaussian approximation to the likelihood is more sensible (Milios et al., 2018).

### The boring details
The transformation of the labels is based on the formalization of a simple intuition, which is the inversion of the softmax transformation. 
Labels are viewed as a set of parameters of a degenerate Dirichlet distribution. 
The degeneracy of the Dirichlet distribution is solved by adding a small regularization, say $\alpha = 0.05$, to the parameters.
At this point, Dirichlet distributed random variables can be constructed as a ratio of Gamma random variables, that is, if $x_i \sim \mathrm{Gamma}(a_i, b)$, then $\frac{x_i}{\sum_j x_j}\sim \mathrm{Dir}(\mathbf{a})$.

The Gamma random variables are approximated with log-Normals by moment matching, which become Gaussian after a logarithm transformation. 
By doing so, we obtain a representation of the labels which allows us to use standard regression with a Gaussian likelihood, and which retrieves an approximate Dirichlet when mapping predictions back using the softmax transformation. 
As a result, the latent functions obtained represent probabilities of class labels.
The only small complication is that the transformation imposes a different noise level for labels that are $0$ or $1$, yielding a heteroskedastic regression model.

### In practice
Given $Y = \{y_i\, |\,  y_i = f(x_i)\}$ where $y_i$ is one-hot encoded, the transformation is defined as:
$$
\text{var}(Y) = \log{[(Y + \alpha)^{-1} + 1]}
$$
$$
\text{mean}(Y) = \log{(Y + \alpha) - \text{var}(Y)/2}
$$
Given this approximation, we place a GP prior over $\mathbf{f}$ and evaluate the posterior over the $C$ latent processes (where $C$ is the number of classes). 
It is possible to make kernel parameters independent across processes, or shared so that they are informed by all classes. 
For simplicity we will share the kernel parameters but you can try to make them independed. 

Let's run a simple experiment.

In [ ]:
set_seed()

# Define dataset
numObs = 25
obs_x = torch.from_numpy(np.sort((np.random.choice(x, numObs, replace=False)))).float()
obs_t = torch.from_numpy(np.stack([np.where(f(obs_x)>=0, 1, 0), np.where(f(obs_x)>=0, 0, 1)]).T).float()
test_x = torch.from_numpy(np.linspace(-30, 85, 1)).float()

fig, (ax0, ax1) = plt.subplots(1, 2, figsize=[5, 2], sharey=True)
ax0.plot(obs_x.numpy(), obs_t.numpy()[:,0], '.', color='black', label='Class 0')
ax1.plot(obs_x.numpy(), obs_t.numpy()[:,1], '.', color='black', label='Class 1')
ax0.set_ylim(-0.05, 1.05)
ax1.set_ylim(-0.05, 1.05)
ax0.legend(loc='lower left')
ax1.legend()
fig.suptitle('Classification problem', y=1.02)
plt.show()

In [ ]:
set_seed()

# Define parameters to optimize
logl = torch.nn.Parameter(torch.tensor(1).float())
logsigmaf = torch.nn.Parameter(torch.tensor(-1).float())

# Just like vanilla PyTorch for DL
optimizer = torch.optim.SGD([logl, logsigmaf, ], lr=0.01, momentum=0.95, nesterov=True)

# Computes Dirichlet transformed variables
alpha = 0.05
obs_t_transf_var = torch.log((obs_t + alpha) ** (-1) + 1 )
obs_t_transf_mean = torch.log(obs_t + alpha) - (obs_t_transf_var) / 2

# Run optimization
for i in range(100):
    optimizer.zero_grad()
    params = [logl.exp(), logsigmaf.exp()]
    _, _, log_lik_0 =  gp_inference(obs_x, obs_t_transf_mean[:,0], test_x, params + [obs_t_transf_var[:,0]]) 
    _, _, log_lik_1 =  gp_inference(obs_x, obs_t_transf_mean[:,1], test_x, params + [obs_t_transf_var[:,1]]) 
    nlog_lik = - log_lik_0 - log_lik_1
    nlog_lik.backward()
    optimizer.step()   

# Run final inference and ...
test_x = torch.from_numpy(np.linspace(-50, 115, 200)).float()
prior_mean = (obs_t_transf_mean.max() + obs_t_transf_mean.min())/2
params = [logl.exp(), logsigmaf.exp()]
posterior_m_0, posterior_v_0, log_lik = gp_inference(obs_x, obs_t_transf_mean[:,0], test_x, params + [obs_t_transf_var[:,0]], prior_mean) 
posterior_m_1, posterior_v_1, log_lik = gp_inference(obs_x, obs_t_transf_mean[:,1], test_x, params + [obs_t_transf_var[:,1]], prior_mean) 
posterior_m = torch.stack([posterior_m_0.detach(), posterior_m_1.detach()]).t()
posterior_std_0 = torch.sqrt(torch.diag(posterior_v_0.detach()))
posterior_std_1 = torch.sqrt(torch.diag(posterior_v_1.detach()))
posterior_std = torch.stack([posterior_std_0.detach(), posterior_std_1.detach()]).t()

In [ ]:
set_seed()
# ... and plot
q = 95
mean_true = np.zeros([posterior_m.shape[0], 2])
lb_true = np.zeros([posterior_m.shape[0], 2])
ub_true = np.zeros([posterior_m.shape[0], 2])
source = np.random.randn(1000, 2)
for i in range(posterior_m.shape[0]):
    samples = source * np.sqrt(posterior_std[i,:].numpy()) + posterior_m[i,:].numpy()
    samples = np.exp(samples) / np.exp(samples).sum(1).reshape(-1, 1)
    Q = np.percentile(samples, [100-q, q], axis=0)
    mean_true[i,:] = samples.mean(0)
    lb_true[i,:] = Q[0,:]
    ub_true[i,:] = Q[1,:]
    
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2, figsize=[9, 5], sharex=True)

ax0.plot(obs_x.numpy(), obs_t_transf_mean[:,0].numpy(), '.', c='black', label='Observations')
ax0.plot(test_x.numpy(), posterior_m[:,0].numpy(), color="grey", label=r'posterior $\mu$')  
ax0.fill_between(test_x.numpy(), (posterior_m[:,0] + posterior_std_0*2).numpy(), (posterior_m[:,0] - posterior_std_0*2).numpy(),
                color="grey", alpha=0.2, label=r'posterior $2\sigma$')
ax0.errorbar(obs_x.numpy(), obs_t_transf_mean[:,0].numpy(), obs_t_transf_var[:,0].sqrt().numpy(),  fmt='.', c='black', alpha=.37)

ax1.plot(obs_x.numpy(), obs_t[:,0].numpy(), '.', c='black', label='Observations')
ax1.plot(test_x.numpy(), mean_true[:,0], color="grey", label=r'posterior $\mu$')  
ax1.fill_between(test_x.numpy(), lb_true[:,0], ub_true[:,0],
                color="grey", alpha=0.2, label=r'posterior $2\sigma$')

ax2.plot(obs_x.numpy(), obs_t_transf_mean[:,1].numpy(), '.', c='black', label='Observations')
ax2.plot(test_x.numpy(), posterior_m[:,1].numpy(), color="grey", label=r'posterior $\mu$')  
ax2.fill_between(test_x.numpy(), (posterior_m[:,1] + posterior_std_1*2).numpy(), (posterior_m[:,1] - posterior_std_1*2).numpy(),
                color="grey", alpha=0.2, label=r'posterior $2\sigma$')
ax2.errorbar(obs_x.numpy(), obs_t_transf_mean[:,1].numpy(), obs_t_transf_var[:,1].sqrt().numpy(),  fmt='.', c='black', alpha=.37)

ax3.plot(obs_x.numpy(), obs_t[:,1].numpy(), '.', c='black', label='Observations')
ax3.plot(test_x.numpy(), mean_true[:,1], color="grey", label=r'posterior $\mu$')  
ax3.fill_between(test_x.numpy(), lb_true[:,1], ub_true[:,1],
                color="grey", alpha=0.2, label=r'posterior $2\sigma$')

ax0.set_title('GP in latent space (class 0)')
ax0.set_ylim(-8,2)
ax1.set_title('GP in original space (class 0)')
ax1.set_ylim(-0.05, 1.05)
ax2.set_title('GP in latent space (class 1)')
ax2.set_ylim(-8,2)
ax3.set_title('GP in latent space (class 1)')
ax3.set_ylim(-0.05, 1.05)
ax3.legend()
plt.show()